## Appendix: Data Collection and Cleaning

I downloaded the Lahman's Baseball Database, which contained a multitude of MLB statistics. The relevant csv files were the Team Data, Salaries, Batting, Pitching, and People ones, containing team aggregate statistics,  players salaries, individual batting and pitching statistics, and player information respectively, organized by year, team, and player (when relevant).

In [1]:
#Load Libraries
import pandas as pd

In [2]:
#Load CSV Files
team_data = pd.read_csv('team_data.csv')
salaries = pd.read_csv('salaries.csv')
batting_data = pd.read_csv('Batting.csv')
pitching_data = pd.read_csv('Pitching.csv')
players = pd.read_csv('People.csv')

Since the salary data only goes from 1985 to 2016, I will limit the data to those years.

In [3]:
team_data = team_data.loc[(team_data['yearID'] >= 1985) & (team_data['yearID'] <= 2016)]
batting_data = batting_data.loc[(batting_data['yearID'] >= 1985) & (batting_data['yearID'] <= 2016)]
pitching_data = pitching_data.loc[(pitching_data['yearID'] >= 1985) & (pitching_data['yearID'] <= 2016)]

Next, I need to filter the salaries Data Frame and then group the team salaries by year and team, and then get the sum to find the each team's payroll by year. This "column" will then be added to the team_data DataFrame.

In [4]:
salaries_copy = salaries[['yearID', 'teamID','salary']].copy()
total_salaries = salaries_copy.groupby(['yearID', 'teamID']).sum()
total_salaries.head()

salary
yearID teamID          
1985   ATL     14807000
       BAL     11560712
       BOS     10897560
       CAL     14427894
       CHA      9846178

Now, I add the total team salary to team_data DataFrame to indicate the total payroll for the team.

In [5]:
total_salaries.reset_index(drop=True, inplace=True)
team_data.reset_index(drop=True, inplace=True)
total_salaries_series = total_salaries['salary']
team_data['TSalary'] = total_salaries_series

I will get the z-score for each team's salary for that season. To do this, I need to get the mean and standard deviation of each season's total salary, adding each as columns for reference.

In [6]:
salaries2 = team_data[['yearID', 'TSalary']]
season_mean = salaries2.groupby(['yearID']).mean()
season_std = salaries2.groupby(['yearID']).std()
season_mean = season_mean.reset_index()
season_std = season_std.reset_index()
series_mean = []
series_std = []
for index in range(len(team_data)):
    year = team_data['yearID'][index]
    mean = season_mean['TSalary'][year-1985]
    series_mean.append(mean)
for index in range(len(team_data)):
    year = team_data['yearID'][index]
    mean = season_std['TSalary'][year-1985]
    series_std.append(mean)

team_data['SalaryMean'] = series_mean
team_data['SalaryStDev'] = series_std
team_data['Z-Score'] = round((team_data['TSalary'] - team_data['SalaryMean'])/(team_data['SalaryStDev']), 4)

Filter the relevant statistics for the analysis on team statistics:

In [7]:
team_data = team_data[['yearID', 'teamID', 'Rank', 'G', 'W', 'L', 'DivWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', 'HR', 'BB', 'RA', 'ER', 'ERA', 'IPouts', 'HA', 'HRA', 'BBA', 'E', 'FP', 'name', 'TSalary', 'Z-Score']]
team_data.head()

,yearID,teamID,Rank,G,W,L,DivWin,LgWin,WSWin,R,...,ERA,IPouts,HA,HRA,BBA,E,FP,name,TSalary,Z-Score
0,1985,ATL,5,162,66,96,N,N,N,632,...,4.19,4372,1512,134,642,159,0.976,Atlanta Braves,14807000,1.9149
1,1985,BAL,4,161,83,78,N,N,N,818,...,4.38,4282,1480,160,568,129,0.979,Baltimore Orioles,11560712,0.6011
2,1985,BOS,5,163,81,81,N,N,N,800,...,4.06,4384,1487,130,540,145,0.977,Boston Red Sox,10897560,0.3327
3,1985,CAL,2,162,90,72,N,N,N,732,...,3.91,4372,1453,171,514,112,0.982,California Angels,14427894,1.7615
4,1985,CHA,3,163,85,77,N,N,N,736,...,4.07,4355,1411,161,569,111,0.982,Chicago White Sox,9846178,-0.0928


For binary outcome variables (division win, League win, Division Win), instead of string 'N' or 'Y', create dummy variables (0 or 1):

In [8]:
team_data['DivWin'] = pd.get_dummies(team_data['DivWin'])['Y']
team_data['LgWin'] = pd.get_dummies(team_data['LgWin'])['Y']
team_data['WSWin'] = pd.get_dummies(team_data['WSWin'])['Y']
team_data.head()

,yearID,teamID,Rank,G,W,L,DivWin,LgWin,WSWin,R,...,ERA,IPouts,HA,HRA,BBA,E,FP,name,TSalary,Z-Score
0,1985,ATL,5,162,66,96,0,0,0,632,...,4.19,4372,1512,134,642,159,0.976,Atlanta Braves,14807000,1.9149
1,1985,BAL,4,161,83,78,0,0,0,818,...,4.38,4282,1480,160,568,129,0.979,Baltimore Orioles,11560712,0.6011
2,1985,BOS,5,163,81,81,0,0,0,800,...,4.06,4384,1487,130,540,145,0.977,Boston Red Sox,10897560,0.3327
3,1985,CAL,2,162,90,72,0,0,0,732,...,3.91,4372,1453,171,514,112,0.982,California Angels,14427894,1.7615
4,1985,CHA,3,163,85,77,0,0,0,736,...,4.07,4355,1411,161,569,111,0.982,Chicago White Sox,9846178,-0.0928


Keep batting and pitching statistics consistent with team data, as well as other relevant statistics:

In [9]:
batting_data = batting_data[['playerID', 'yearID', 'teamID', 'G', 'AB', 'R', 'H', 'HR', 'RBI', 'BB']]
pitching_data = pitching_data[['playerID', 'yearID', 'teamID', 'W', 'L', 'G', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'BAOpp', 'ERA', 'R']]
batting_data.reset_index(drop=True, inplace=True)
pitching_data.reset_index(drop=True, inplace=True)

To get better name keeping, use the playerIDs in batting and pitching DataFrames to add their name an additional column, pulling from the players file. Also, add their salary for that season (if not in DataFrame, add as NaN).

In [10]:
batter_names = []
batter_salary = []
for index in range(len(batting_data)):
    row = batting_data.iloc[index:(index+1)].copy()
    row.reset_index(drop=True, inplace=True)
    playerID = row['playerID'][0]
    yearID = row['yearID'][0]
    player = players.loc[players['playerID'] == playerID, ['nameFirst', 'nameLast']].copy()
    salary_info = salaries.loc[(salaries['playerID'] == playerID) & (salaries['yearID'] == yearID), ['salary']].copy()
    player.reset_index(drop=True, inplace=True)
    salary_info.reset_index(drop=True, inplace=True)
    name = player['nameFirst'][0] + ' ' + player['nameLast'][0]
    try:
        salary = salary_info['salary'][0]
    except:
        salary = float('NaN')
    batter_names.append(name)
    batter_salary.append(salary)

In [11]:
pitcher_names = []
pitcher_salary = []
for index in range(len(pitching_data)):
    row = pitching_data.iloc[index:(index+1)].copy()
    row.reset_index(drop=True, inplace=True)
    playerID = row['playerID'][0]
    yearID = row['yearID'][0]
    player = players.loc[players['playerID'] == playerID, ['nameFirst', 'nameLast']].copy()
    salary_info = salaries.loc[(salaries['playerID'] == playerID) & (salaries['yearID'] == yearID), ['salary']].copy()
    player.reset_index(drop=True, inplace=True)
    salary_info.reset_index(drop=True, inplace=True)
    name = player['nameFirst'][0] + ' ' + player['nameLast'][0]
    try:
        salary = salary_info['salary'][0]
    except:
        salary = float('NaN')
    pitcher_names.append(name)
    pitcher_salary.append(salary)

In [12]:
batting_data['Salary'] = batter_salary
batting_data['Name'] = batter_names
pitching_data['Salary'] = pitcher_salary
pitching_data['Name'] = pitcher_names

Organize columns to wanted order.

In [13]:
batting_data = batting_data[['Name', 'playerID', 'yearID', 'teamID', 'G', 'AB', 'R', 'H', 'HR', 'RBI', 'BB', 'Salary']]
pitching_data = pitching_data[['Name', 'playerID', 'yearID', 'teamID', 'W', 'L', 'G', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'BAOpp', 'ERA', 'R', 'Salary']]

In [14]:
batting_data.head()

,Name,playerID,yearID,teamID,G,AB,R,H,HR,RBI,BB,Salary
0,Don Aase,aasedo01,1985,BAL,54,0,0,0,0,0.0,0,NaN
1,Johnny Abrego,abregjo01,1985,CHN,6,9,0,0,0,1.0,0,NaN
2,Jim Acker,ackerji01,1985,TOR,61,0,0,0,0,0.0,0,170000.0
3,Ricky Adams,adamsri02,1985,SFN,54,121,12,23,2,10.0,5,NaN
4,Juan Agosto,agostju01,1985,CHA,54,0,0,0,0,0.0,0,147500.0


In [15]:
pitching_data.head()

,Name,playerID,yearID,teamID,W,L,G,SV,IPouts,H,ER,HR,BB,BAOpp,ERA,R,Salary
0,Don Aase,aasedo01,1985,BAL,10,6,54,14,264,83,37,6,35,0.258,3.78,44,NaN
1,Johnny Abrego,abregjo01,1985,CHN,1,1,6,0,72,32,17,3,12,0.352,6.38,18,NaN
2,Jim Acker,ackerji01,1985,TOR,7,2,61,10,259,86,31,7,43,0.268,3.23,35,170000.0
3,Juan Agosto,agostju01,1985,CHA,4,3,54,1,181,45,24,3,23,0.210,3.58,27,147500.0
4,Rick Aguilera,aguilri01,1985,NYN,10,7,21,0,367,118,44,8,37,0.258,3.24,49,NaN


Output cleaned DataFrames to csv files.

In [16]:
compression_opts_1 = dict(method='zip', archive_name='team_data_final.csv')
compression_opts_2 = dict(method='zip', archive_name='batting_data_final.csv')
compression_opts_3 = dict(method='zip', archive_name='pitching_data_final.csv')
team_data.to_csv('team_data_final.zip', compression=compression_opts_1)
batting_data.to_csv('batting_data_final.zip', compression=compression_opts_2)
pitching_data.to_csv('pitching_data_final.zip', compression=compression_opts_3)